# MEng Computer Science Thesis Implementation

**Title:** Benchmarking Persian LLMs Versus State-of-the-Art NLP Models for Sentiment Analysis <br/>
**Student Name:** Seyyed Jalal Tabatabaee <br/>
**Student ID:** GH1033801 <br/>
**Supervisor:** Prof. Dr. Reza Babaei <br/>
**Semester:** Summer 2025 <br/>
**University:** Gisma University of Applied Sciences <br/>
**Github Link:** https://github.com/shayantabatabaee/llm-sentiment-analysis <br/>
**Dataset Link:** https://github.com/persiannlp/parsinlu



### Introduction

This project focuses on sentiment analysis for the Persian language, a low-resource language that has been underexplored in modern NLP research. While most sentiment analysis studies have been conducted in English, Persian poses unique challenges due to its complex grammar, diverse vocabulary, and the absence of diacritics.

Traditional approaches to sentiment analysis involve steps like text preprocessing, embedding generation, and classification using machine learning (ML) or deep learning (DL) models. However, with the emergence of large language models (LLMs) such as ChatGPT, new techniques like zero-shot and few-shot prompting have shown promising results — even without fine-tuning.

This research benchmarks Persian LLMs (with fewer parameters) on the ParsiNLU sentiment dataset and compares their performance against state-of-the-art traditional models like BERT. The goal is to investigate whether smaller, Persian-specific LLMs can offer performance comparable to larger multilingual models while being more cost-efficient and practical for real-world applications.



### Install Libraries

In [ ]:
# !pip install -q datasets
# !pip install -q pandas
# !pip install -q torch
# !pip install -q transformers
# !pip install -U -q accelerate
# !pip install -q tqdm
# !pip install -q scikit-learn
# !pip install -q seaborn
# !pip install -q numpy

### Import Libraries

In [ ]:
import os

os.environ["FORCE_TF_AVAILABLE"] = "0"
os.environ["USE_TF"]="0"

In [ ]:
import re
import copy
import math
import torch
import transformers
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from IPython.display import display, display_html
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LlamaTokenizer, LlamaForCausalLM
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

### Initialize Variables

In [ ]:
DATASET_URL = "https://raw.githubusercontent.com/persiannlp/parsinlu/master/data/sentiment-analysis/food.jsonl"
LABEL_MAP = {
    "3": "OTHER",
    "2": "POSITIVE",
    "1": "POSITIVE",
    "0": "NEUTRAL",
    "-1": "NEGATIVE",
    "-2": "NEGATIVE"
}
TEST_COUNT = 190
RANDOM_STATE = 18
os.environ['HF_TOKEN'] = UserSecretsClient().get_secret("HF_TOKEN")

In [ ]:
label_en_fa = {
    "OTHER": "سایر",
    "POSITIVE": "مثبت",
    "NEUTRAL": "خنثی",
    "NEGATIVE": "منفی"
}
label_fa_en = {v:k for k,v in label_en_fa.items()}

## Dataset Preprocessing

To ensure a direct comparison with the baseline study [(Abaskohi et al., 2024)](https://arxiv.org/pdf/2404.02403), we used the same dataset and fixed the random seed. Additionally, we merged the original six sentiment categories into four, matching the label structure used in the baseline. The original dataset contains six labels: *VERY_POSITIVE, POSITIVE, NEUTRAL, NEGATIVE, VERY_NEGATIVE,* and *OTHER*. For our analysis, we merged *VERY_POSITIVE* with *POSITIVE* and *VERY_NEGATIVE* with *NEGATIVE*.

### Load Dataset

In [ ]:
dataset = load_dataset("json", data_files=DATASET_URL)
df = dataset['train'].to_pandas()
df = df[['review', 'sentiment']]
df['sentiment'] = df['sentiment'].map(LABEL_MAP)

In [ ]:
sub_df = df.sample(n=TEST_COUNT, random_state=RANDOM_STATE)

### Visualize Dataset

In [ ]:
def insight(data):
  print(f"Dataset shape: {data.shape}", end="\n\n")
  print("Five sample of data")
  display(data.head(5))
  print("\n\nDataset info", end="\n\n")
  display(data['sentiment'].value_counts())
  data['sentiment'].value_counts().plot.bar(x="Sentiment", y="Count", rot=45)

In [ ]:
insight(df)

In [ ]:
insight(sub_df)

## Prompts

In [ ]:
def get_message(prompt):
  return [{"role": "system", "content": prompt[0]},{"role": "user", "content": prompt[1]}]

### System Prompts

In [ ]:
system_prompt_01 = """
تو یک دستیار هوشمند برای تحلیل احساسات هستی.
یک نظر کوتاه درباره ی غذا به زبان فارسی دریافت میکنی.
وظیفه‌ی تو این است که احساس کلی نظر را در یکی از چهار دسته‌ی زیر طبقه‌بندی کنی:

- مثبت
- منفی
- خنثی
- سایر

فقط یکی از این چهار برچسب را برگردان. هیچ متن اضافی، توضیح، یا عبارتی مانند «احساس:» در پاسخ ننویس.
اگر احساس نظر نامشخص، متناقض یا ترکیبی بود، فقط برچسب «سایر» را برگردان
اگر حس متن مثبت یا منفی نبود خنثی را برگردان
"""

In [ ]:
system_prompt_02 = """
You are an AI assistant for sentiment analysis.
You will receive a short user opinion about food, written in Persian.
Your task is to classify the overall sentiment of the opinion as one of the following four categories:

- POSITIVE
- NEGATIVE
- NEUTRAL
- OTHER

Return ONLY one of the four labels listed above. Do not include any additional text, explanations, or prefixes such as 'Sentiment:'.
If the sentiment is unclear, contradictory, or mixed, return OTHER.
If the sentiment is not POSITIVE or NEGATIVE return NEUTRAL.
"""

### Zero-Shot User Prompts

In [ ]:
zs_user_prompt_01= """
نظر: REVIEW
احساس (مثبت، منفی، خنثی، سایر):
"""

In [ ]:
zs_user_prompt_02="""
Review: REVIEW
Sentiment (POSITIVE, NEGATIVE, NEUTRAL, OTHER):
"""

### Few-Shot User Prompts

In [ ]:
diff_df = df.loc[~df.index.isin(sub_df.index)]
def create_examples_df(df, n_samples):
    groups = df['sentiment'].unique()
    sampled = []

    for sentiment in groups:
        group_df = df[df['sentiment'] == sentiment]
        sampled_df = group_df.sample(n=n_samples, random_state=RANDOM_STATE)
        sampled.append(sampled_df)

    return pd.concat(sampled).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

In [ ]:
def create_example_prompts(n_samples):
  examples_df = create_examples_df(diff_df, n_samples)
  en_examples = ""
  fa_examples = ""
  for i, row in examples_df.iterrows():
    en_examples += f"Example {i+1}:\n"
    en_examples += f"Review: {row['review']}\n"
    en_examples += f"Sentiment: {row['sentiment']}\n"

    fa_examples += f"مثال {i+1}:\n"
    fa_examples += f"نظر: {row['review']}\n"
    fa_examples += f"احساس: {label_en_fa[row['sentiment']]}\n"

  return en_examples, fa_examples

#### One-Shot User Prompts

In [ ]:
en_1_examples, fa_1_examples = create_example_prompts(1)
print(en_1_examples)
print("*" * 50)
print(fa_1_examples)

In [ ]:
fs_1_user_prompts_01 = f"""
مثالها:
{fa_1_examples}

نظر: REVIEW
احساس (مثبت، منفی، خنثی، سایر):
"""

In [ ]:
fs_1_user_prompts_02 = f"""
Examples:
{en_1_examples}

Review: REVIEW
Sentiment (POSITIVE, NEGATIVE, NEUTRAL, OTHER):
"""

#### Three-Shot User Prompts

In [ ]:
en_3_examples, fa_3_examples = create_example_prompts(3)

In [ ]:
fs_3_user_prompts_01 = f"""
مثالها:
{fa_3_examples}

نظر: REVIEW
احساس (مثبت، منفی، خنثی، سایر):
"""

In [ ]:
fs_3_user_prompts_02 = f"""
Examples:
{en_3_examples}

Review: REVIEW
Sentiment (POSITIVE, NEGATIVE, NEUTRAL, OTHER):
"""

### Create Prompts

In [ ]:
zero_shot_combination = [(system_prompt_01, zs_user_prompt_01), (system_prompt_02, zs_user_prompt_02)]
few_shot_combination = [
    (system_prompt_01, fs_1_user_prompts_01), 
    (system_prompt_02, fs_1_user_prompts_02),
    (system_prompt_01, fs_3_user_prompts_01),
    (system_prompt_02, fs_3_user_prompts_02)]
zero_shot_final_prompts = [get_message(prompt) for prompt in zero_shot_combination]
few_shot_final_prompts = [get_message(prompt) for prompt in few_shot_combination]
prompts = {
    "zs_fa": zero_shot_final_prompts[0],
    "zs_en": zero_shot_final_prompts[1],
    "few_1_fa": few_shot_final_prompts[0],
    "few_1_en": few_shot_final_prompts[1],
    "few_3_fa": few_shot_final_prompts[2],
    "few_3_en": few_shot_final_prompts[3]
}

## Experiments

In [ ]:
def generate_result(df, run_func ,tokenizer, model):

    df.loc[:, 'zs_fa'] = None
    df.loc[:, 'zs_en'] = None
    df.loc[:, 'few_1_fa'] = None
    df.loc[:, 'few_1_en'] = None
    df.loc[:, 'few_3_fa'] = None
    df.loc[:, 'few_3_en'] = None

    total_steps = len(df) * len(prompts)

    with tqdm(total=total_steps, desc="Processing") as pbar:
        for index, row in df.iterrows():
            copy_prompts = copy.deepcopy(prompts)
            for k, v in copy_prompts.items():
                v[1]['content'] = v[1]['content'].replace("REVIEW", row['review'])
                result = run_func(v, tokenizer, model)
                df.at[index, k] = result
                pbar.update(1)

In [ ]:
def cleanup():
    import torch    
    import gc
    import subprocess

    subprocess.run(["rm", "-rf", "/root/.cache/huggingface"], check=True)
    
    while gc.collect()!=0:
        for i in range(0,3):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            gc.collect()

### PartAI/Dorna2-Llama3.1-8B-Instruct

In [ ]:
def part_run(prompt, tokenizer, model):
    model.generation_config.temperature=None
    model.generation_config.top_p=None

    input_ids = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        return_tensors="pt",
        return_attention_mask=True
    ).to(model.device)

    pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    attention_mask = (input_ids != pad_token_id).long()

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        #attention_mask=attention_mask,
        pad_token_id=pad_token_id,
        max_new_tokens=5,
        eos_token_id=terminators,
        do_sample=False,        # deterministic classification
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
model_1_result = sub_df.copy()

In [ ]:
model_path = "PartAI/Dorna2-Llama3.1-8B-Instruct"
tokenizer_part_8b = AutoTokenizer.from_pretrained(model_path)
model_part_8b = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_1_result, part_run, tokenizer_part_8b, model_part_8b)

In [ ]:
model_1_result.to_csv('/kaggle/working/part_dorna_8b.csv')
model_1_result.head(20)

In [ ]:
del part_run
del tokenizer_part_8b
del model_part_8b
cleanup()

### universitytehran/PersianMind-v1.0

In [ ]:
def remove_brackets_content(text):
    text = re.sub(r'\s*\([^)]*\)', '', text)
    return text.rstrip('\n')

def ut_run(prompt, tokenizer, model):
    model.generation_config.temperature=None
    model.generation_config.top_p=None

    TEMPLATE = "{context}\n{prompt}"
    CONTEXT = prompt[0]['content']
    PROMPT = remove_brackets_content(prompt[1]['content'])

    model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
    input_tokens = tokenizer(model_input, return_tensors="pt")
    input_tokens = input_tokens.to(model.device)
    generate_ids = model.generate(**input_tokens,
                                  max_new_tokens=1024,
                                  do_sample=False,
                                  repetition_penalty=1.1)
    model_output = tokenizer.batch_decode(generate_ids,
                                          skip_special_tokens=True,
                                          clean_up_tokenization_spaces=False)[0]
    return model_output[len(model_input):].strip()

In [ ]:
model_2_result = sub_df.copy()

In [ ]:
model_path = "universitytehran/PersianMind-v1.0"
tokenizer_ut_7b = AutoTokenizer.from_pretrained(model_path)
model_ut_7b = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_2_result, ut_run, tokenizer_ut_7b, model_ut_7b)

In [ ]:
model_2_result.to_csv('/kaggle/working/ut_mind_7b.csv')
model_2_result.head(20)

In [ ]:
del ut_run
del tokenizer_ut_7b
del model_ut_7b
cleanup()

### CohereLabs/aya-expanse-8b

In [ ]:
def aya_run(prompt, tokenizer, model):
    input_ids = tokenizer.apply_chat_template(prompt,
                                              tokenize=True,
                                              add_generation_prompt=True,
                                              return_tensors="pt").to(model.device)

    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=5,
        do_sample=False
        )

    generated = gen_tokens[0][input_ids.shape[-1]:]
    gen_text = tokenizer.decode(generated, skip_special_tokens=True)
    return gen_text.strip()

In [ ]:
model_3_result = sub_df.copy()

In [ ]:
model_path = "CohereLabs/aya-expanse-8b"
tokenizer_aya_8b = AutoTokenizer.from_pretrained(model_path)
model_aya_8b = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_3_result, aya_run, tokenizer_aya_8b, model_aya_8b)

In [ ]:
model_3_result.to_csv('/kaggle/working/cohere_aya_8b.csv')
model_3_result.head(20)

In [ ]:
del aya_run
del tokenizer_aya_8b
del model_aya_8b
cleanup()

### MaralGPT/Maral-7B-alpha-1

In [ ]:
def remove_brackets_content(text):
    text = re.sub(r'\s*\([^)]*\)', '', text)
    return text.rstrip('\n')

def maral_run(prompt, tokenizer, model):
    TEMPLATE = "### Human:{context}{prompt}"
    CONTEXT = prompt[0]['content']
    PROMPT = remove_brackets_content(prompt[1]['content'])

    model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
    inputs = tokenizer(model_input, return_tensors="pt").to(model.device)

    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=5,
        pad_token_id=tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(model_input):].strip()

In [ ]:
model_4_result = sub_df.copy()

In [ ]:
model_path = "MaralGPT/Maral-7B-alpha-1"
tokenizer_maral_7b = AutoTokenizer.from_pretrained(model_path)
model_maral_7b = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_4_result, maral_run, tokenizer_maral_7b, model_maral_7b)

In [ ]:
model_4_result.to_csv('/kaggle/working/maral_7b.csv')
model_4_result.head(20)

In [ ]:
del maral_run
del tokenizer_maral_7b
del model_maral_7b
cleanup()

### MehdiHosseiniMoghadam/AVA-Llama-3-V2

In [ ]:
def remove_brackets_content(text):
    text = re.sub(r'\s*\([^)]*\)', '', text)
    return text.rstrip('\n')

def ava_run(prompt, tokenizer, model):
    TEMPLATE = "### Human:{context}\n ###{prompt}"
    CONTEXT = prompt[0]['content']
    PROMPT = remove_brackets_content(prompt[1]['content'])

    model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
    inputs = tokenizer(model_input, return_tensors="pt").to(model.device)

    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=5,
        pad_token_id=tokenizer.eos_token_id
    )

    outputs = model.generate(**inputs, generation_config=generation_config)
    outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(model_input):].strip().split('\n')[0]
    return outputs

In [ ]:
model_5_result = sub_df.copy()

In [ ]:
model_path = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"
tokenizer_ava_8b = AutoTokenizer.from_pretrained(model_path)
model_ava_8b = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_5_result, ava_run, tokenizer_ava_8b, model_ava_8b)

In [ ]:
model_5_result.to_csv('/kaggle/working/ava_8b.csv')
model_5_result.head(20)

In [ ]:
del ava_run
del tokenizer_ava_8b
del model_ava_8b
cleanup()

### ViraIntelligentDataMining/PersianLLaMA-13B-Instruct

In [ ]:
def remove_brackets_content(text):
    text = re.sub(r'\s*\([^)]*\)', '', text)
    return text.rstrip('\n')

def persian_llama_run(prompt, tokenizer, model):
    TEMPLATE = "### Instruction:{context}{prompt}"
    CONTEXT = prompt[0]['content']
    PROMPT = remove_brackets_content(prompt[1]['content'])

    model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
    inputs = tokenizer(model_input, return_tensors="pt").to(model.device)

    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=5,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    outputs = model.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(model_input):].strip()

In [ ]:
model_6_result = sub_df.copy()

In [ ]:
model_path = "ViraIntelligentDataMining/PersianLLaMA-13B-Instruct"
tokenizer_persian_llama_13b = LlamaTokenizer.from_pretrained(model_path)
model_persian_llama_13b = LlamaForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
generate_result(model_6_result, persian_llama_run, tokenizer_persian_llama_13b, model_persian_llama_13b)

In [ ]:
model_6_result.to_csv('/kaggle/working/persian_llama_13b.csv')
model_6_result.head(20)

In [ ]:
del persian_llama_run
del tokenizer_persian_llama_13b
del model_persian_llama_13b
cleanup()

## Result

In [ ]:
# model_1_result = pd.read_csv('/kaggle/input/result-sub/part_dorna_8b.csv', index_col=[0])
# model_2_result = pd.read_csv('/kaggle/input/result-sub/ut_mind_7b.csv', index_col=[0])
# model_3_result = pd.read_csv('/kaggle/input/result-sub/cohere_aya_8b.csv', index_col=[0])
# model_4_result = pd.read_csv('/kaggle/input/result-sub/maral_7b.csv', index_col=[0])
# model_5_result = pd.read_csv('/kaggle/input/result-sub/ava_8b.csv', index_col=[0])
# model_6_result = pd.read_csv('/kaggle/input/result-sub/persian_llama_13b.csv', index_col=[0])

In [ ]:
def map_sentiment(df):
    def standardize_value(value):
        if value.upper() in label_en_fa:
            return value
        elif value in label_fa_en:
            return label_fa_en[value]
        else:
            return "-"

    experiment_keys = list(prompts.keys())
    for exp in experiment_keys:
        df[exp] = df[exp].apply(standardize_value)

In [ ]:
def plot_confusion_matrix(df):
    labels = ['POSITIVE', 'NEGATIVE', 'NEUTRAL', 'OTHER']
    prediction_columns = list(prompts.keys())

    num_preds = len(prediction_columns)
    cols = 2
    rows = (num_preds + 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 5 * rows))
    axes = axes.flatten()

    for i, pred_col in enumerate(prediction_columns):
        cm = confusion_matrix(df['sentiment'], df[pred_col], labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)

        sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', ax=axes[i], cbar=False)
        axes[i].set_title(f'Confusion Matrix: {pred_col}')
        axes[i].set_xlabel('Predicted')
        axes[i].set_ylabel('True')

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
def calculate_metrics(df):
    labels = ['POSITIVE', 'NEGATIVE', 'NEUTRAL', 'OTHER']
    prediction_columns = list(prompts.keys())

    results = {}

    for i, pred_col in enumerate(prediction_columns):
        report = classification_report(df['sentiment'], df[pred_col], labels=labels, output_dict=True, zero_division=0)

        report["accuracy"] = {
            "precision": accuracy_score(df['sentiment'], df[pred_col]),
            "recall": accuracy_score(df['sentiment'], df[pred_col]),
            "f1-score": accuracy_score(df['sentiment'], df[pred_col]),
            "support": len(df['sentiment'])
        }

        report_df = pd.DataFrame(report).transpose()
        report_df = report_df.round(3)
        results[pred_col] = report_df

    return results

In [ ]:
def show_metrics_grid(df):
    metrics_dict = calculate_metrics(df)
    dfs = list(metrics_dict.values())
    titles = list(metrics_dict.keys())

    html_tables = [
        f"<div style='padding:10px'><h4 style='text-align:center'>Experiment: {title}</h4>{df.to_html()}</div>"
        for title, df in zip(titles, dfs)
    ]

    rows = [''.join(html_tables[::2]), ''.join(html_tables[1::2])]

    grid_html = '<table><tr><td>' + '</td><td>'.join(rows) + '</td></tr></table>'
    display_html(grid_html, raw=True)

### PartAI/Dorna2-Llama3.1-8B-Instruct

In [ ]:
map_sentiment(model_1_result)

In [ ]:
plot_confusion_matrix(model_1_result)

In [ ]:
show_metrics_grid(model_1_result)

### universitytehran/PersianMind-v1.0

In [ ]:
map_sentiment(model_2_result)

In [ ]:
plot_confusion_matrix(model_2_result)

In [ ]:
show_metrics_grid(model_2_result)

### CohereLabs/aya-expanse-8b

In [ ]:
map_sentiment(model_3_result)

In [ ]:
plot_confusion_matrix(model_3_result)

In [ ]:
show_metrics_grid(model_3_result)

### MaralGPT/Maral-7B-alpha-1

In [ ]:
map_sentiment(model_4_result)

In [ ]:
plot_confusion_matrix(model_4_result)

In [ ]:
show_metrics_grid(model_4_result)

### MehdiHosseiniMoghadam/AVA-Llama-3-V2

In [ ]:
map_sentiment(model_5_result)

In [ ]:
plot_confusion_matrix(model_5_result)

In [ ]:
show_metrics_grid(model_5_result)

### ViraIntelligentDataMining/PersianLLaMA-13B-Instruct

In [ ]:
map_sentiment(model_6_result)

In [ ]:
plot_confusion_matrix(model_6_result)

In [ ]:
show_metrics_grid(model_6_result)

## Conclusion

In [ ]:
def get_best_experiment(metrics):
    result = {}
    for k, v in metrics.items():
        best_accuracy = 0
        best = None
        key = ""
        for _, experiment in v.items():
            if experiment.loc['weighted avg']['f1-score'] > best_accuracy:
                best = experiment
                best_accuracy = experiment.loc['weighted avg']['f1-score']
                key = _
        result[k+'-'+key] = {}
        result[k+'-'+key]['best'] = best
    return result

In [ ]:
metrics = {
    "PartAI/Dorna2-Llama3.1-8B-Instruct": calculate_metrics(model_1_result),
    "universitytehran/PersianMind-v1.0": calculate_metrics(model_2_result),
    "CohereLabs/aya-expanse-8b": calculate_metrics(model_3_result),
    "MaralGPT/Maral-7B-alpha-1": calculate_metrics(model_4_result),
    "MehdiHosseiniMoghadam/AVA-Llama-3-V2": calculate_metrics(model_5_result),
    "ViraIntelligentDataMining/PersianLLaMA-13B-Instruct": calculate_metrics(model_6_result)
}

In [ ]:
best_for_each_model = get_best_experiment(metrics)

### Radar Chart

In [ ]:
def plot_radar_chart(data, title, key, ax):
    metrics = ['POSITIVE',
               'NEGATIVE',
               'NEUTRAL',
               'OTHER',
               'macro avg',
               'weighted avg',
               'accuracy']

    N = len(metrics)
    theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
    theta = np.concatenate([theta, [theta[0]]])

    ax.plot(subplot_kw={'projection': 'polar'})

    ax.set_title(title, y=1.15, fontsize=20)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(90)
    ax.spines['polar'].set_zorder(1)
    ax.spines['polar'].set_color('lightgrey')

    color_palette = [
        '#FF0000',
        '#0000FF',
        '#FFD700',
        '#FF8C00',
        '#008000',
        '#9467bd']

    idx = 0

    for model_name, metric in data.items():
        experiment = metric[key]
        values = experiment.loc[metrics]['f1-score'].to_list()
        values = values + [values[0]]
        ax.plot(theta, values, linewidth=1.75, linestyle='solid', label=model_name, marker='o', markersize=10, color=color_palette[idx % len(color_palette)])
        #ax.fill(theta, values, alpha=0.50, color=color_palette[idx % len(color_palette)])
        idx+=1

    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_yticklabels(["0", "20", "40", "60", "80", "100"], color="black", size=12)

    ax.set_xticks(theta)
    ax.set_xticklabels(metrics+[metrics[0]], color='black', size=12)
    return fig

### All Experiments

In [ ]:
keys = list(prompts.keys())

num_charts = len(keys)
cols = 2
rows = math.ceil(num_charts / cols)

fig, axs = plt.subplots(rows, cols, subplot_kw=dict(polar=True), figsize=(12, 5 * rows))

axs = axs.reshape(rows, cols) if rows > 1 else [axs]

for idx, key in enumerate(keys):
    row, col = divmod(idx, cols)
    ax = axs[row][col]
    plot_radar_chart(metrics, f"Experiment: {key}", key, ax)

for idx in range(len(keys), rows * cols):
    row, col = divmod(idx, cols)
    fig.delaxes(axs[row][col])

handles, labels = axs[0][0].get_legend_handles_labels()

fig.legend(
    handles,
    labels,
    loc='upper center',
    bbox_to_anchor=(0.5, -0.01),
    fontsize=12,
    frameon=False
)

plt.tight_layout()
plt.show()

## Best Model

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(polar=True), figsize=(6, 6))
plot_radar_chart(best_for_each_model, "Experiment: Best Model", 'best' ,ax)
ax.legend(loc='upper right', bbox_to_anchor=(2.1, 1.1))
plt.show()